# Goal-Based Data Collection
A practical approach to robot reinforcement learning is to first collect a large batch of real or simulated robot interaction data, 
using some data collection policy, and then learn from this data to perform various tasks, using offline learning algorithms.

In this notebook, we will demonstrate how to collect diverse dataset for a simple robotics manipulation task
using the algorithms detailed in the following paper:
[Efficient Self-Supervised Data Collection for Offline Robot Learning](https://arxiv.org/abs/2105.04607).

Presets with predefined parameters for all three algorithms shown in the paper can be found here:

* Random Agent: ```presets/RoboSuite_CubeExp_Random.py```

* Intrinsic Reward Agent: ```presets/RoboSuite_CubeExp_TD3_Intrinsic_Reward.py```

* Goal-Based Agent: ```presets/RoboSuite_CubeExp_TD3_Goal_Based.py```

You can run those presets using the command line:

`coach -p RoboSuite_CubeExp_TD3_Goal_Based`


## Preliminaries
First, get the required imports and other general settings we need for this notebook.


In [ ]:
from rl_coach.agents.td3_exp_agent import TD3GoalBasedAgentParameters
from rl_coach.architectures.embedder_parameters import InputEmbedderParameters
from rl_coach.architectures.layers import Dense, Conv2d, BatchnormActivationDropout, Flatten
from rl_coach.base_parameters import EmbedderScheme
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps
from rl_coach.environments.robosuite_environment import RobosuiteGoalBasedExpEnvironmentParameters, \
    OptionalObservations
from rl_coach.filters.filter import NoInputFilter, NoOutputFilter
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import ScheduleParameters
from rl_coach.architectures.head_parameters import RNDHeadParameters
from rl_coach.schedules import LinearSchedule


Then, we define the training schedule for the agent. `improve_steps` dictates the number of samples in the final data-set.


In [ ]:
####################
# Graph Scheduling #
####################

schedule_params = ScheduleParameters()
schedule_params.improve_steps = TrainingSteps(300000)
schedule_params.steps_between_evaluation_periods = TrainingSteps(300000)
schedule_params.evaluation_steps = EnvironmentEpisodes(0)
schedule_params.heatup_steps = EnvironmentSteps(1000)



In this example, we will be using the goal-based algorithm for data-collection. Therefore, we populate
the `TD3GoalBasedAgentParameters` class with our desired algorithm specific parameters.

The goal-based data collected is based on TD3, using this class you can change the TD3 specific parameters as well.

A detailed description of the goal-based and TD3 algorithm specific parameters can be found in 
```agents/td3_exp_agent.py``` and ```agents/td3_agent.py``` respectively.


In [ ]:
#########
# Agent #
#########

agent_params = TD3GoalBasedAgentParameters()
agent_params.algorithm.use_non_zero_discount_for_terminal_states = False
agent_params.algorithm.identity_goal_sample_rate = 0.04
agent_params.exploration.noise_schedule = LinearSchedule(1.5, 0.5, 300000)

agent_params.algorithm.rnd_sample_size = 2000
agent_params.algorithm.rnd_batch_size = 500
agent_params.algorithm.rnd_optimization_epochs = 4
agent_params.algorithm.td3_training_ratio = 1.0
agent_params.algorithm.identity_goal_sample_rate = 0.0
agent_params.algorithm.img_obs_key = 'camera'
agent_params.algorithm.replay_buffer_save_steps = 25000
agent_params.algorithm.replay_buffer_save_path = './tutorials'

agent_params.input_filter = NoInputFilter()
agent_params.output_filter = NoOutputFilter()


Next, we'll define the networks' architecture and parameters as they appear in the paper.


In [ ]:
# Camera observation pre-processing network scheme
camera_obs_scheme = [
    Conv2d(32, 8, 4),
    BatchnormActivationDropout(activation_function='relu'),
    Conv2d(64, 4, 2),
    BatchnormActivationDropout(activation_function='relu'),
    Conv2d(64, 3, 1),
    BatchnormActivationDropout(activation_function='relu'),
    Flatten(),
    Dense(256),
    BatchnormActivationDropout(activation_function='relu')
]

obs_name = 'obs-goal'

# Actor
actor_network = agent_params.network_wrappers['actor']
actor_network.input_embedders_parameters = {
    'measurements': InputEmbedderParameters(scheme=EmbedderScheme.Empty),
    obs_name: InputEmbedderParameters(scheme=camera_obs_scheme, activation_function='none')
}

actor_network.middleware_parameters.scheme = [Dense(300), Dense(200)]
actor_network.learning_rate = 1e-4

# Critic
critic_network = agent_params.network_wrappers['critic']
critic_network.input_embedders_parameters = {
    'action': InputEmbedderParameters(scheme=EmbedderScheme.Empty),
    'measurements': InputEmbedderParameters(scheme=EmbedderScheme.Empty),
    obs_name: InputEmbedderParameters(scheme=camera_obs_scheme, activation_function='none')
}

critic_network.middleware_parameters.scheme = [Dense(400), Dense(300)]
critic_network.learning_rate = 1e-4

# RND
agent_params.network_wrappers['predictor'].input_embedders_parameters = \
    {'camera': InputEmbedderParameters(scheme=EmbedderScheme.Empty,
                                       input_rescaling={'image': 1.0},
                                       flatten=False)}
agent_params.network_wrappers['constant'].input_embedders_parameters = \
    {'camera': InputEmbedderParameters(scheme=EmbedderScheme.Empty,
                                       input_rescaling={'image': 1.0},
                                       flatten=False)}
agent_params.network_wrappers['predictor'].heads_parameters = [RNDHeadParameters(is_predictor=True)]


The last thing we need to define is the environment parameters for the manipulation task.
This environment is a 7DoF Franka Panda robotic arm with a closed gripper and cartesian
position control of the end-effector. The robot is positioned on a table, and a cube object with colored sides is placed in
front of it.


In [ ]:
###############
# Environment #
###############
env_params = RobosuiteGoalBasedExpEnvironmentParameters(level='CubeExp')
env_params.robot = 'Panda'
env_params.custom_controller_config_fpath = './rl_coach/environments/robosuite/osc_pose.json'
env_params.base_parameters.optional_observations = OptionalObservations.CAMERA
env_params.base_parameters.render_camera = 'frontview'
env_params.base_parameters.camera_names = 'agentview'
env_params.base_parameters.camera_depths = False
env_params.base_parameters.horizon = 200
env_params.base_parameters.ignore_done = False
env_params.base_parameters.use_object_obs = True
env_params.frame_skip = 1
env_params.base_parameters.control_freq = 2
env_params.base_parameters.camera_heights = 84
env_params.base_parameters.camera_widths = 84
env_params.extra_parameters = {'hard_reset': False}



Finally, we create the graph manager and call `graph_manager.improve()` in order to start the data collection.


In [ ]:
graph_manager = BasicRLGraphManager(agent_params=agent_params, env_params=env_params, schedule_params=schedule_params)
graph_manager.improve()


Once the data collection is complete, the data-set will saved to path specified by `agent_params.algorithm.replay_buffer_save_path`.

At this point, the data can be used to learn any downstream task you define on that environment. the script below
shows a visualization of the data-set. The dots represent a position of the cube on the table as seen in the data-set, 
and the color corresponds to the color of the face at the top. The number at the top signifies that number of dots a plot contains for a certain color.


In [ ]:
import os
import numpy as np
import joblib
from collections import OrderedDict
from enum import IntEnum
from pylab import subplot
from gym.envs.robotics.rotations import quat2euler, mat2euler, quat2mat
import matplotlib.pyplot as plt


class CubeColor(IntEnum):
    YELLOW = 0
    CYAN = 1
    WHITE = 2
    RED = 3
    GREEN = 4
    BLUE = 5
    UNKNOWN = 6


x_range = [-0.3, 0.3]
y_range = [-0.3, 0.3]

COLOR_MAP = OrderedDict([
    (int(CubeColor.YELLOW), 'yellow'),
    (int(CubeColor.CYAN), 'cyan'),
    (int(CubeColor.WHITE), 'white'),
    (int(CubeColor.RED), 'red'),
    (int(CubeColor.GREEN), 'green'),
    (int(CubeColor.BLUE), 'blue'),
    (int(CubeColor.UNKNOWN), 'black'),
])

# Mapping between (subset of) euler angles to top face color, based on the initial cube rotation
COLOR_ROTATION_MAP = OrderedDict([
    (CubeColor.YELLOW, (0, 2, [np.array([0, 0]),
                               np.array([np.pi, np.pi]), np.array([-np.pi, -np.pi]),
                               np.array([-np.pi, np.pi]), np.array([np.pi, -np.pi])])),
    (CubeColor.CYAN, (0, 2, [np.array([0, np.pi]), np.array([0, -np.pi]),
                             np.array([np.pi, 0]), np.array([-np.pi, 0])])),
    (CubeColor.WHITE, (1, 2, [np.array([-np.pi / 2])])),
    (CubeColor.RED, (1, 2, [np.array([np.pi / 2])])),
    (CubeColor.GREEN, (0, 2, [np.array([np.pi / 2, 0])])),
    (CubeColor.BLUE, (0, 2, [np.array([-np.pi / 2, 0])])),
])


def get_cube_top_color(cube_quat, atol):
    euler = mat2euler(quat2mat(cube_quat))
    for color, (start_dim, end_dim, xy_rotations) in COLOR_ROTATION_MAP.items():
        if any(list(np.allclose(euler[start_dim:end_dim], xy_rotation, atol=atol) for xy_rotation in xy_rotations)):
            return color
    return CubeColor.UNKNOWN


def pos2cord(x, y):
    x = max(min(x, x_range[1]), x_range[0])
    y = max(min(y, y_range[1]), y_range[0])
    x = int(((x - x_range[0])/(x_range[1] - x_range[0]))*99)
    y = int(((y - y_range[0])/(y_range[1] - y_range[0]))*99)
    return x, y


rb_path = os.path.join('./Resources', 'RB_TD3GoalBasedAgent.joblib.bz2')
episodes = joblib.load(rb_path)

pos_idx = 25
quat_idx = 28
positions = []
colors = []
for episode in episodes:
    for transition in episode:
        x, y = transition.state['measurements'][pos_idx:pos_idx+2]
        positions.append([x, y])
        angle = quat2euler(transition.state['measurements'][quat_idx:quat_idx+4])
        colors.append(int(get_cube_top_color(transition.state['measurements'][quat_idx:quat_idx+4], np.pi / 4)))

        x_cord, y_cord = pos2cord(x, y)

    x, y = episode[-1].next_state['measurements'][pos_idx:pos_idx+2]
    positions.append([x, y])
    colors.append(int(get_cube_top_color(episode[-1].next_state['measurements'][quat_idx:quat_idx+4], np.pi / 4)))

    x_cord, y_cord = pos2cord(x, y)


fig = plt.figure()
axes = []
for j in range(6):
    axes.append(subplot(1, 6, j + 1))
    xy = np.array(positions)[np.array(colors) == list(COLOR_MAP.keys())[j]]
    axes[-1].scatter(xy[:, 1], xy[:, 0], c=COLOR_MAP[j], alpha=0.01, edgecolors='black')
    plt.xlim(y_range)
    plt.ylim(x_range)
    plt.xticks([])
    plt.yticks([])
    title = 'N=' + str(xy.shape[0])
    plt.title(title)

for ax in axes:
    ax.invert_yaxis()

plt.show()
